In [1]:
import torch
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)

from bert_util import bert_tokenize_data, tensor_train_test_split, train_bert_model, model_predict, get_data_loader
import pandas as pd
from util import eda_augment_dataset, get_dataframe_from_json
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import classification_report

/home/emiliano/Desktop/Università/Human Language Technologies/Project/HLT Project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-21 18:59:55.328457: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 18:59:55.335225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747846795.343445   16451 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered


In [2]:
train_df = pd.read_json('../train_dataset.json', lines=False)
test_df = pd.read_json('../test_dataset.json', lines=False)
train_df

,turn,utterance,emotion,act,hat
0,3,"I'll take one, too.",happiness,inform,0
1,8,"You know, we are superior to other clothes com...",no_emotion,inform,3
2,5,"Her new boyfriend, right?",no_emotion,commissive,1
3,9,How about recommending him to use the storage ...,no_emotion,directive,4
4,1,"Oh, a bouquet of flowers. It's very kind of you.",surprise,commissive,1
...,...,...,...,...,...
808,0,I prefer potatoes to eggplants.,no_emotion,inform,0
809,0,"Mr. Smith, I would like to get right to the po...",no_emotion,question,1
810,4,Yeah?,no_emotion,question,1
811,0,I am so bored all day.,no_emotion,inform,0


In [3]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
token_ids, attention_masks = bert_tokenize_data(tokenizer, train_df['utterance'].values)
train_dataloader, val_dataloader = tensor_train_test_split(torch.tensor(train_df['hat'].values), token_ids, attention_masks, test_size=0.1)

In [4]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
epochs = 10
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

model = train_bert_model(model, optimizer, scheduler, train_dataloader, val_dataloader, epochs)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-------------------- Epoch 1 --------------------

Training:
---------
Start Time:       2025-05-21 19:00:05.844753
Average Loss:     1.3238509621309198
Time Taken:       0:00:11.459815

Validation:
---------
Start Time:       2025-05-21 19:00:17.304937
Average Loss:     1.4233713150024414
Average Accuracy: 0.4090909090909091
Time Taken:       0:00:00.375181

-------------------- Epoch 2 --------------------

Training:
---------
Start Time:       2025-05-21 19:00:17.680473
Average Loss:     1.1604303681980008
Time Taken:       0:00:11.280846

Validation:
---------
Start Time:       2025-05-21 19:00:28.961643
Average Loss:     1.4465822712941603
Average Accuracy: 0.5227272727272727
Time Taken:       0:00:00.380521

-------------------- Epoch 3 --------------------

Training:
---------
Start Time:       2025-05-21 19:00:29.342597
Average Loss:     0.940253781883613
Time Taken:       0:00:11.298843

Validation:
---------
Start Time:       2025-05-21 19:00:40.641800
Average Loss:     1.427

In [6]:

test_texts = test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(tokenizer, serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = test_df['hat'].values.flatten()
accuracy = np.sum(preds == labels_flat) / len(labels_flat)
preds_array = np.array(preds)

hat_map = {
    0: "red",
    1: "white",
    2: "black",
    3: "yellow",
    4: "green",
}

print(classification_report(labels_flat, preds_array, target_names=list(hat_map.values())))

              precision    recall  f1-score   support

         red       0.49      0.45      0.47        40
       white       0.74      0.75      0.75       110
       black       0.43      0.43      0.43        23
      yellow       0.35      0.32      0.33        19
       green       0.27      0.33      0.30        12

    accuracy                           0.59       204
   macro avg       0.46      0.46      0.46       204
weighted avg       0.59      0.59      0.59       204



In [8]:
augmented_train_df = pd.read_json('../eda_train_dataset.json', lines=False)
augmented_train_df

,turn,utterance,emotion,act,hat
0,3,"I'll take one, too.",happiness,inform,0
1,8,"You know, we are superior to other clothes com...",no_emotion,inform,3
2,5,"Her new boyfriend, right?",no_emotion,commissive,1
3,9,How about recommending him to use the storage ...,no_emotion,directive,4
4,1,"Oh, a bouquet of flowers. It's very kind of you.",surprise,commissive,1
...,...,...,...,...,...
3738,0,i am wholly so bored all day,no_emotion,inform,0
3739,0,i am so bored all daylight,no_emotion,inform,0
3740,0,i am so bored all day,no_emotion,inform,0
3741,0,i am so bored all daytime,no_emotion,inform,0


In [9]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
token_ids, attention_masks = bert_tokenize_data(tokenizer, augmented_train_df['utterance'].values)
train_dataloader, val_dataloader = tensor_train_test_split(torch.tensor(augmented_train_df['hat'].values), token_ids, attention_masks, test_size=0.1)


model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
epochs = 10
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

model = train_bert_model(model, optimizer, scheduler, train_dataloader, val_dataloader, epochs)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-------------------- Epoch 1 --------------------

Training:
---------
Start Time:       2025-05-21 19:06:31.134568
Average Loss:     0.8155961225778219
Time Taken:       0:00:51.257857

Validation:
---------
Start Time:       2025-05-21 19:07:22.392744
Average Loss:     1.2723816867838516
Average Accuracy: 0.5227963525835867
Time Taken:       0:00:01.541842

-------------------- Epoch 2 --------------------

Training:
---------
Start Time:       2025-05-21 19:07:23.934986
Average Loss:     0.2834358179268168
Time Taken:       0:00:51.502900

Validation:
---------
Start Time:       2025-05-21 19:08:15.438223
Average Loss:     2.9783911045561444
Average Accuracy: 0.44034954407294835
Time Taken:       0:00:01.559063

-------------------- Epoch 3 --------------------

Training:
---------
Start Time:       2025-05-21 19:08:16.997662
Average Loss:     0.1705245903423255
Time Taken:       0:00:52.321096

Validation:
---------
Start Time:       2025-05-21 19:09:09.319149
Average Loss:     1.1

In [10]:
test_texts = test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(tokenizer, serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = test_df['hat'].values.flatten()
accuracy = np.sum(preds == labels_flat) / len(labels_flat)
preds_array = np.array(preds)

print(classification_report(labels_flat, preds_array, target_names=list(hat_map.values())))

              precision    recall  f1-score   support

         red       0.54      0.47      0.51        40
       white       0.66      0.84      0.74       110
       black       0.36      0.22      0.27        23
      yellow       0.54      0.37      0.44        19
       green       0.33      0.08      0.13        12

    accuracy                           0.61       204
   macro avg       0.49      0.40      0.42       204
weighted avg       0.57      0.61      0.58       204

